In [2]:
import requests
import pandas as pd
import numpy as np
import time
pd.set_option('display.max_columns', 200)

In [3]:
# 매번 사용한 api_key값과 header를 생성한다.
import requests
 
api_key = "RGAPI-568a4862-9059-44de-b5ee-8b30fce0e037"
header = {"X-Riot-Token" : api_key}

In [4]:
# 첼린저,그랜드마스터,마스터의 솔로랭크 정보를 가져오기 위한 함수 생성

def league_v4_tier(tier):
    # 솔랭 기준
    queue = 'RANKED_SOLO_5x5'
 
    if tier == 'challenger' :
        url = f"https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{queue}"
    elif tier == 'grandmaster':
        url = f"https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{queue}"
    else:
        url = f"https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{queue}"
        
    return requests.get(url, headers=header)

In [18]:
challenger_data = pd.DataFrame(league_v4_tier('challenger').json())

In [19]:
# 첼린저 티어의 솔로랭크 정보중 entries에 들어 있는 summonerId 및 승패정보를 테이블로 담기

challenger_df = pd.DataFrame([challenger_data['entries'][0]])
data_size = len(challenger_data)

for i in range(1,data_size):
    append_df = pd.DataFrame([challenger_data['entries'][i]])
    challenger_df = pd.concat([challenger_df,append_df],ignore_index = True)

challenger_df['tier'] = 'CHALLENGER'
challenger_df

,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,tier
0,6ATFBrxtyjUc8Bp5-QuRuGPAalJBKddPiVDe5vnKDDbQkx...,1497,I,219,159,True,False,False,False,CHALLENGER
1,4XkYCh-Xiblsoqrds__yEKU1Z_yD9qUeXET6wiXQAZ5uPV4,1156,I,88,45,False,False,False,False,CHALLENGER
2,h9w1Hz2po2fR9Yjsc1Jnb5J5C6sk9sCt0NYf9ol7jnpWnk...,1065,I,111,68,False,False,True,False,CHALLENGER
3,LCO5dHGZnSmoruxg7F4KxB8OiTKXYPWLbSS3q_EyVRyLygI,1114,I,226,185,False,False,True,True,CHALLENGER
4,-Cl3yxRZcmxOX79SmdUbqxl5bgo0xOkS5I-UglRQ1t3WOK...,1228,I,116,66,False,False,False,False,CHALLENGER
...,...,...,...,...,...,...,...,...,...,...
295,38CcZcqif29j0MKitMeULoxB_v8aP5mZ0oN7QrSeie9h7do,1428,I,177,121,True,False,False,True,CHALLENGER
296,SNbToq-anQICrwjJgzlEyEcbrsDtHBV5qLYz1tl3ifNhrSk,1181,I,249,203,False,False,False,False,CHALLENGER
297,6YubbsfmcRmTTsGflA_K3ZvbhWt4k4eiva-719yw9iKmK80,1237,I,297,233,True,False,False,False,CHALLENGER
298,uvoJSS0Ibz4oc7hBK-BkrQO-GwXPGN4KkWNMjU3THNGYUkA,1082,I,210,174,False,False,False,False,CHALLENGER


In [5]:
# summonerid로 소환사 정보인 puuid를 가져오기 위한 API코드
def summoners(encryptedSummonerId):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{encryptedSummonerId}"
    
    return requests.get(url, headers = header)

In [24]:
# challenger_df에 추가적으로 컬럼을 생성하고 temp값을 넣어 둔다.(주석처리해둔다)

challenger_df['accountId'] = 'temp_1'
challenger_df['puuid'] = 'temp_2'
challenger_df['profileIconId'] = 'temp_3'
challenger_df['revisionDate'] = 'temp_4'
challenger_df['summonerLevel'] = 'temp_5'

In [25]:
# 유저마다 puuid,accountid등을 가져와서 top_class_df에 넣는다.(첼린저만 가져오는 코드)
# 해당코드는 한번 생성한 뒤에는 실행되지 않게끔 주석처리해둔다.

for i in range(len(challenger_df)):
    encryptedSummonerId = challenger_df['summonerId'].loc[i]
    summoners_info = summoners(encryptedSummonerId).json()

    acid = summoners_info['accountId']
    puid = summoners_info['puuid']
    proid = summoners_info['profileIconId']
    re_date = summoners_info['revisionDate']
    summo_level = summoners_info['summonerLevel']

    challenger_df.loc[i,'accountId'] = acid
    challenger_df.loc[i,'puuid'] = puid
    challenger_df.loc[i,'profileIconId'] = proid
    challenger_df.loc[i,'revisionDate'] = re_date
    challenger_df.loc[i,'summonerLevel'] = summo_level
    
    print(i,summoners_info['id'])
    if i%100 == 0:
        challenger_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_summoner_challenger_tb.csv")
        print("중간저장 합니다.")
    else:
        pass

    time.sleep(1.5)

challenger_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\lol_summoner_challenger_tb.csv")
challenger_df

0 6ATFBrxtyjUc8Bp5-QuRuGPAalJBKddPiVDe5vnKDDbQkxbiA8KwGGtv-Q
중간저장 합니다.
1 4XkYCh-Xiblsoqrds__yEKU1Z_yD9qUeXET6wiXQAZ5uPV4
2 h9w1Hz2po2fR9Yjsc1Jnb5J5C6sk9sCt0NYf9ol7jnpWnk-bMDWdtJnRUg
3 LCO5dHGZnSmoruxg7F4KxB8OiTKXYPWLbSS3q_EyVRyLygI
4 -Cl3yxRZcmxOX79SmdUbqxl5bgo0xOkS5I-UglRQ1t3WOKUXx1fQNmXFKg
5 -CrAai6unneQSnB30Dl_RPG4Fns_HZzpB70a1_PrLurnePTrt9GKsok7iQ
6 SfKzYwPxxlNZ1Ief-T98PoErzAemRZo8zDdJNMxwFDSFoQI
7 wR65FIcrUa_5RIC8oQ51UcWNfb4QXfmraK5tBjz1YNUsfk8
8 Ow2yvsw7pShj6ptuQtvQLHW-K1UycODwEcd7Y_n7xbKoyJM
9 cHqD5t-tD5fB9oXuVubxyjZj4tBByS-fGJA5JUAy4qCgix8
10 8O2qaPTxMdRmNAa68SXSXeNb1skAyLMoEKt8OQ6kOwNRXxg
11 0gveSgWCRQ97lfPK8Ip5od_qBHvjKGOfsl33JdFmz2DFJVc
12 cerWLtyeN8XPSmFKqniEUFvX9r9KfRST_ddR7f4tilYulZZmftlBxGH0Cw
13 RTA2JJbjFZlZ22r9RDwUzEhohwjxRwJx5DH9g0DpXPRyn4I
14 WM1bxfvRj9Di-t-yq3lAVo3yXPtdxiLmXZZpdk799FaGHqE
15 FBwrdSqebPVce3wKt2kpjJ4VcdBninKRrGaLaJdtfc2hDFM
16 x0ClhJFGaVjAmANJ2JsF2J4-e2yVUIpC5vRQhAgLMVdDCK03H1Af9YYCbA
17 GyUf3oQBf2hAMZpqttRsU2VrHafYTDAuAM1kYijochG8kg
18 vhV92XivAu1Ehh

,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,tier,accountId,puuid,profileIconId,revisionDate,summonerLevel
0,6ATFBrxtyjUc8Bp5-QuRuGPAalJBKddPiVDe5vnKDDbQkx...,1497,I,219,159,True,False,False,False,CHALLENGER,6CaAJvBIxTnoy-XNn0lSC2DurKC0AkwHq5jdcYP74VVClN...,Hj4IKQP9akZANxxbWGTj8qDpuTedj2LaSYmDhoJlRu5nWj...,6467,1722965401857,238
1,4XkYCh-Xiblsoqrds__yEKU1Z_yD9qUeXET6wiXQAZ5uPV4,1156,I,88,45,False,False,False,False,CHALLENGER,j5qHhr_o4egkh9LGuOxCRNc9ldxCV0wtmStuHOm9dYLf7r...,sajCpaF7BH74dhWPB77IpAGXJ_laznrOWP7PP8UD8Mxml8...,4022,1721991818000,265
2,h9w1Hz2po2fR9Yjsc1Jnb5J5C6sk9sCt0NYf9ol7jnpWnk...,1065,I,111,68,False,False,True,False,CHALLENGER,O5aurQYgtSoXX706dPeXioAzCzdI8lscF1sN1fhJiLCh63...,pnSAn41ICv43WXuw6fvEppjfEn50zh94qw-PQfKgol9Oew...,1425,1722959464755,66
3,LCO5dHGZnSmoruxg7F4KxB8OiTKXYPWLbSS3q_EyVRyLygI,1114,I,226,185,False,False,True,True,CHALLENGER,OqOwoLunNlCCMDJtpIGXOFYzG7O_TDssLK_f-zB00Kyedyk,kyg6218vaWqvD7WGQwV_Ie4rNRNMc35vdys5_8jW0CNWXk...,6296,1722792574000,1345
4,-Cl3yxRZcmxOX79SmdUbqxl5bgo0xOkS5I-UglRQ1t3WOK...,1228,I,116,66,False,False,False,False,CHALLENGER,ItmuhSurwj6JK7ogk4HaTC8SEnHqys1sNIS1YTYSo-LDHV...,_5V7GJB_pYw3SwOCYuGcDfKPo6a-erdKSLdagOr9swZbC3...,907,1722627465000,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,38CcZcqif29j0MKitMeULoxB_v8aP5mZ0oN7QrSeie9h7do,1428,I,177,121,True,False,False,True,CHALLENGER,OD7UPXnV4xXh2m5Q1NwYgPo-v20X9e1l1VHc5DLgNd4pJos,sHTQFZl8EQlmqd8w-9iKMyo1Z28jvyNzJTUwQkkyeGc2wT...,3860,1722965449460,552
296,SNbToq-anQICrwjJgzlEyEcbrsDtHBV5qLYz1tl3ifNhrSk,1181,I,249,203,False,False,False,False,CHALLENGER,dbWKbD5juPeEmODq2LTQyk1Ejwj5VvWTcr2O5zYE6_tOfCQ,kjR5o3_UQpliFt3XaCmbOQGHZt5f6GAIncLm6Pvn4OwT0v...,6580,1722392163000,855
297,6YubbsfmcRmTTsGflA_K3ZvbhWt4k4eiva-719yw9iKmK80,1237,I,297,233,True,False,False,False,CHALLENGER,y2qPJglo9t1e1qpiyG-sB7rWd-OKtywc08K1rf-8um-ulIQ,fopmjQCmoiVWLUUndi9PBU-riUYqQI-1K_lLtlNVKWlZBP...,4568,1722967321309,130
298,uvoJSS0Ibz4oc7hBK-BkrQO-GwXPGN4KkWNMjU3THNGYUkA,1082,I,210,174,False,False,False,False,CHALLENGER,Hv884ijgAvYz1SrYePUCi0IVxpMyM2p3gGdKa3GqNB8GbA...,rgBywR_Pb5TTvi498hJ0D0KTAcCD35DK523k4upikqhqtk...,4533,1722967267910,717


In [10]:
# challenger데 tb를 불러온다
challenger_user_basic_df = pd.read_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_summoner_tb.csv")
challenger_user_basic_df

,Unnamed: 0,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,tier,accountId,puuid,profileIconId,revisionDate,summonerLevel
0,0,6ATFBrxtyjUc8Bp5-QuRuGPAalJBKddPiVDe5vnKDDbQkx...,1497,I,219,159,True,False,False,False,CHALLENGER,6CaAJvBIxTnoy-XNn0lSC2DurKC0AkwHq5jdcYP74VVClN...,Hj4IKQP9akZANxxbWGTj8qDpuTedj2LaSYmDhoJlRu5nWj...,6467,1722965401857,238
1,1,4XkYCh-Xiblsoqrds__yEKU1Z_yD9qUeXET6wiXQAZ5uPV4,1156,I,88,45,False,False,False,False,CHALLENGER,j5qHhr_o4egkh9LGuOxCRNc9ldxCV0wtmStuHOm9dYLf7r...,sajCpaF7BH74dhWPB77IpAGXJ_laznrOWP7PP8UD8Mxml8...,4022,1721991818000,265
2,2,h9w1Hz2po2fR9Yjsc1Jnb5J5C6sk9sCt0NYf9ol7jnpWnk...,1065,I,111,68,False,False,True,False,CHALLENGER,O5aurQYgtSoXX706dPeXioAzCzdI8lscF1sN1fhJiLCh63...,pnSAn41ICv43WXuw6fvEppjfEn50zh94qw-PQfKgol9Oew...,1425,1722959464755,66
3,3,LCO5dHGZnSmoruxg7F4KxB8OiTKXYPWLbSS3q_EyVRyLygI,1114,I,226,185,False,False,True,True,CHALLENGER,OqOwoLunNlCCMDJtpIGXOFYzG7O_TDssLK_f-zB00Kyedyk,kyg6218vaWqvD7WGQwV_Ie4rNRNMc35vdys5_8jW0CNWXk...,6296,1722792574000,1345
4,4,-Cl3yxRZcmxOX79SmdUbqxl5bgo0xOkS5I-UglRQ1t3WOK...,1228,I,116,66,False,False,False,False,CHALLENGER,ItmuhSurwj6JK7ogk4HaTC8SEnHqys1sNIS1YTYSo-LDHV...,_5V7GJB_pYw3SwOCYuGcDfKPo6a-erdKSLdagOr9swZbC3...,907,1722627465000,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,38CcZcqif29j0MKitMeULoxB_v8aP5mZ0oN7QrSeie9h7do,1428,I,177,121,True,False,False,True,CHALLENGER,OD7UPXnV4xXh2m5Q1NwYgPo-v20X9e1l1VHc5DLgNd4pJos,sHTQFZl8EQlmqd8w-9iKMyo1Z28jvyNzJTUwQkkyeGc2wT...,3860,1722965449460,552
296,296,SNbToq-anQICrwjJgzlEyEcbrsDtHBV5qLYz1tl3ifNhrSk,1181,I,249,203,False,False,False,False,CHALLENGER,dbWKbD5juPeEmODq2LTQyk1Ejwj5VvWTcr2O5zYE6_tOfCQ,kjR5o3_UQpliFt3XaCmbOQGHZt5f6GAIncLm6Pvn4OwT0v...,6580,1722392163000,855
297,297,6YubbsfmcRmTTsGflA_K3ZvbhWt4k4eiva-719yw9iKmK80,1237,I,297,233,True,False,False,False,CHALLENGER,y2qPJglo9t1e1qpiyG-sB7rWd-OKtywc08K1rf-8um-ulIQ,fopmjQCmoiVWLUUndi9PBU-riUYqQI-1K_lLtlNVKWlZBP...,4568,1722967321309,130
298,298,uvoJSS0Ibz4oc7hBK-BkrQO-GwXPGN4KkWNMjU3THNGYUkA,1082,I,210,174,False,False,False,False,CHALLENGER,Hv884ijgAvYz1SrYePUCi0IVxpMyM2p3gGdKa3GqNB8GbA...,rgBywR_Pb5TTvi498hJ0D0KTAcCD35DK523k4upikqhqtk...,4533,1722967267910,717


In [6]:
# puuid를 활용하여 match list를 불러오는 함수

def match_list(puuid):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start={0}&count={100}"
    
    return requests.get(url, headers=header)

In [29]:
# puuid를 사용하여 유저별로 100개의 match list를 불러오고 중복된 리스트는 제거하여 14,773명의 전체 경기 리스트를 생성(챌린저유저만,주석처리)

challenger_match_list = []

for i in range(len(challenger_user_basic_df['puuid'])):
    puuid = challenger_user_basic_df['puuid'].loc[i]
    add_match_list = match_list(puuid).json()
    challenger_match_list = list(set(challenger_match_list + add_match_list))
    print(i,puuid,len(challenger_match_list))
    time.sleep(1.5)

challenger_match_list

0 Hj4IKQP9akZANxxbWGTj8qDpuTedj2LaSYmDhoJlRu5nWjufCZuBABETPzsyBsxQczlQHWDvlChUmw 100
1 sajCpaF7BH74dhWPB77IpAGXJ_laznrOWP7PP8UD8Mxml8uD3grStwULEyQc-9yBUMWAibbwk9hsJA 200
2 pnSAn41ICv43WXuw6fvEppjfEn50zh94qw-PQfKgol9Oew1xyCcBHvccqe27JWkKMWHIA_fKdBbCkw 300
3 kyg6218vaWqvD7WGQwV_Ie4rNRNMc35vdys5_8jW0CNWXkmOo09Jx_CbjPCvDNcJ2yQFIElUUEdBkw 398
4 _5V7GJB_pYw3SwOCYuGcDfKPo6a-erdKSLdagOr9swZbC3hTKL_VPGoirNICNunNMcKPusnxJODZww 496
5 6vwvPdjrELmATg4Rmo1eHEbB-qXNkpUgrC1NP8epiEpPhOyU_ZkH68cmwz_wdb8UByyLUVmriNsGjQ 595
6 y39CWZ6UY77mGvyTyy9krgeNogxRzYI-2xuXmO3TPApRGR91WR-SgS7UsRcdhwVH2IZLRXPqyUghXg 694
7 hkVT5VMM2zuHX_jChjvkco0T3dwjOFyI7E7IEPSBE1oVTOPumbfsvtSYMhHucwN5rPDZQbGaa1lmTA 794
8 -LxLwePJAwrGZcuTd_EUE2k-YGWbBuaLEvPm3LkQSe6PPMON6BjMmzbfcDcZUiYvb19wTRWbkYdakw 890
9 o_OTUsIBq4kU7QAlXA_8SlNyrEt0cMFVwhkAcfwtVn7LegHIfSZjVXuLqS7A_qHkqJKQ-YTtUDUhow 976
10 v8-5RUj-X_zOM72bO8uxOSPm_Ygej2sQqcm9WdwuqGKeTh7xSP-UelBRAZQEynrVrOZ0Blt_KipEjA 1073
11 KuaCLgPToECYue8XoZmh4J_Oyf6zOm3acjGnUsT6IpMDsUG_eUw5iSWY3Z04

['KR_7135102038',
 'KR_7140864351',
 'KR_7167209159',
 'KR_7167851140',
 'KR_7161586250',
 'KR_7108957850',
 'KR_7141814163',
 'KR_7125183658',
 'KR_7186019293',
 'KR_7199052633',
 'KR_7171530455',
 'KR_7202180224',
 'KR_7184464587',
 'KR_7171631508',
 'KR_7135017903',
 'KR_7140103464',
 'KR_7136548359',
 'KR_7130424019',
 'KR_7204009052',
 'KR_7199595381',
 'KR_7142981000',
 'KR_7197437843',
 'KR_7205928315',
 'KR_7191127711',
 'KR_7146865698',
 'KR_7203902323',
 'KR_7144795231',
 'KR_7170325959',
 'KR_7145013003',
 'KR_7149891796',
 'KR_7195690388',
 'KR_7196959010',
 'KR_7183816366',
 'KR_7141488305',
 'KR_7206639491',
 'KR_7195347572',
 'KR_7126421915',
 'KR_7133042971',
 'KR_7178578050',
 'KR_7195976982',
 'KR_7148009732',
 'KR_7174601182',
 'KR_7198110133',
 'KR_7148463926',
 'KR_7130230359',
 'KR_7190914733',
 'KR_7207074015',
 'KR_7140495190',
 'KR_7114596728',
 'KR_7155972745',
 'KR_7131052107',
 'KR_7155001314',
 'KR_7145595160',
 'KR_7174965851',
 'KR_7191045174',
 'KR_71264

In [7]:
# puuid로 생성한 challenger match list를 테이블형태로 변환하여 CSV파일로 저장(주석처리)

match_df = pd.DataFrame(challenger_match_list)
match_df.rename(columns={0 : 'user_match_list'},inplace=True)
match_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_match_tb.csv")
match_df

NameError: name 'challenger_match_list' is not defined

In [8]:
# 저장된 challenger user의 match list 불러오기

challenger_user_match_df = pd.read_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_match_tb.csv")
challenger_user_match_df = challenger_user_match_df.drop(['Unnamed: 0'],axis=1)
challenger_user_match_df

,user_match_list
0,KR_7135102038
1,KR_7140864351
2,KR_7167209159
3,KR_7167851140
4,KR_7161586250
...,...
14214,KR_7172180171
14215,KR_7206092276
14216,KR_7189380962
14217,KR_7144198833


In [9]:
def match(matchId):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}"
    
    return requests.get(url, headers=header)

In [10]:
len(challenger_user_match_df)

14219

In [13]:
match_data_participants_df = pd.DataFrame()
match_data_teams_df = pd.DataFrame()
match_data_info_df = pd.DataFrame()

for i in range(len(challenger_user_match_df)):
    # matchid를 통해 match정보를 가져오기
    matchId = challenger_user_match_df['user_match_list'].loc[i]
    respones_result = match(matchId)
    match_data_respones = respones_result.status_code
    match_data_row = respones_result.json()
    print("응답상태 : ",match_data_respones)

    if match_data_respones == 200:
        # match정보중 participants를 전처리하여 match별로 쌓아서 테이블로 생성----------------------------------------------
        match_data_participants = pd.DataFrame(match_data_row['info']['participants'])
        match_data_participants['matchId'] = matchId
        match_data_participants_df = pd.concat([match_data_participants_df,match_data_participants],ignore_index = True)
        print(i,'participants생성')
        

        # match정보중 teams를 전처리하여 match별로 쌓아서 테이블로 생성--------------------------------
        match_data_teams = pd.DataFrame(match_data_row['info']['teams'])
        match_data_teams['matchId'] = matchId
        match_data_teams_df = pd.concat([match_data_teams_df,match_data_teams],ignore_index = True)
        print(i,'teams 생성')


        # match정보중 info를 전처리하여 match별로 쌓아서 테이블로 생성------------------------------
        del match_data_row['info']['participants']
        del match_data_row['info']['teams']
        match_data_info = pd.DataFrame(match_data_row['info'],index=[0])
        match_data_info['matchId'] = matchId
        match_data_info_df = pd.concat([match_data_info_df,match_data_info],ignore_index = True)
        print(i,'info 생성')
    else:
        pass
    print(i,matchId)
    if i%200 == 0:
        match_data_participants_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_participants_tb.csv")
        match_data_teams_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_teams_tb.csv")
        match_data_info_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_info_tb.csv")
        print(i,"까지 중간저장 합니다.")
    else:
        pass
    time.sleep(1.5)

match_data_participants_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_participants_tb.csv")
match_data_teams_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_teams_tb.csv")
match_data_info_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_info_tb.csv")

응답상태 :  200
0 participants생성
0 teams 생성
0 info 생성
0 KR_7135102038
0 까지 중간저장 합니다.
응답상태 :  200
1 participants생성
1 teams 생성
1 info 생성
1 KR_7140864351
응답상태 :  200
2 participants생성
2 teams 생성
2 info 생성
2 KR_7167209159
응답상태 :  200
3 participants생성
3 teams 생성
3 info 생성
3 KR_7167851140
응답상태 :  200
4 participants생성
4 teams 생성
4 info 생성
4 KR_7161586250
응답상태 :  200
5 participants생성
5 teams 생성
5 info 생성
5 KR_7108957850
응답상태 :  200
6 participants생성
6 teams 생성
6 info 생성
6 KR_7141814163
응답상태 :  200
7 participants생성
7 teams 생성
7 info 생성
7 KR_7125183658
응답상태 :  200
8 participants생성
8 teams 생성
8 info 생성
8 KR_7186019293
응답상태 :  200
9 participants생성
9 teams 생성
9 info 생성
9 KR_7199052633
응답상태 :  200
10 participants생성
10 teams 생성
10 info 생성
10 KR_7171530455
응답상태 :  200
11 participants생성
11 teams 생성
11 info 생성
11 KR_7202180224
응답상태 :  200
12 participants생성
12 teams 생성
12 info 생성
12 KR_7184464587
응답상태 :  200
13 participants생성
13 teams 생성
13 info 생성
13 KR_7171631508
응답상태 :  200
14 participants생성
14 teams 생성
14 

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
match_data_metadata_df = pd.DataFrame()

# match정보중 metadata를 전처리하여 match별로 쌓아서 테이블로 생성--------------------------------------
for num in range(len(match_data_row['metadata']['participants'])):
    player_number = 'player_' + str(num+1)
    match_data_row['metadata'][str(player_number)] = match_data_row['metadata']['participants'][num]
del(match_data_row['metadata']['participants'])

match_data_metadata = pd.DataFrame(match_data_row['metadata'],index=[0])
match_data_metadata_df = pd.concat([match_data_metadata_df,match_data_metadata],ignore_index = True)

In [ ]:
match_data_metadata_df.to_csv("c:\\Users\\user\\Documents\\GitHub\\data_tb_file\\challenger_metadata_tb_2.csv")